In [21]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error
from math import sqrt


### DATA LOADING:

Set up data model:
(Might need to recreate some of the dataframes from 1.02)

In [22]:
df = pd.read_csv("C:/Users/chris/Documents/Senior_Project/CT_SeniorProject/CrashData_Repo1/data/interim/filtered_dataset.csv")
df_ord = pd.read_csv("C:/Users/chris/Documents/Senior_Project/CT_SeniorProject/CrashData_Repo1/data/external/ORD_Weather.csv")
df_mdw = pd.read_csv("C:/Users/chris/Documents/Senior_Project/CT_SeniorProject/CrashData_Repo1/data/external/Midway_Weather.csv")

# create a df to aggreagate the number of crashes per day
df_crashes_per_day = df.groupby("CRASH_DATE").size().reset_index(name="Total_Crashes")
print(df_crashes_per_day.head())



#print(df.head())
#print(df_ord.head())
#df_ord.info()


#df_ord.shape()
#df_ord.info()     # Overview of column types and missing values
#df_ord.describe()   # Summary statistics for numeric columns
#df_ord.isnull().sum()

   CRASH_DATE  Total_Crashes
0  2018-01-01            337
1  2018-01-02            452
2  2018-01-03            383
3  2018-01-04            360
4  2018-01-05            373


In [26]:
#print(df_ord.head())
#print(df_ord.info())
print(df_mdw.info())
print(df_ord.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2557 entries, 0 to 2556
Data columns (total 48 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   STATION          2557 non-null   object 
 1   NAME             2557 non-null   object 
 2   LATITUDE         2557 non-null   float64
 3   LONGITUDE        2557 non-null   float64
 4   ELEVATION        2557 non-null   float64
 5   DATE             2557 non-null   object 
 6   AWND             2556 non-null   float64
 7   AWND_ATTRIBUTES  2556 non-null   object 
 8   PGTM             826 non-null    float64
 9   PGTM_ATTRIBUTES  826 non-null    object 
 10  PRCP             2553 non-null   float64
 11  PRCP_ATTRIBUTES  2553 non-null   object 
 12  SNOW             207 non-null    float64
 13  SNOW_ATTRIBUTES  207 non-null    object 
 14  SNWD             594 non-null    float64
 15  SNWD_ATTRIBUTES  594 non-null    object 
 16  TAVG             0 non-null      float64
 17  TAVG_ATTRIBUTE

### DATA CLEANING

Need to rename the columns in the ORD weather dataset for easy understanding.

In [ ]:
# Define a dictionary to rename columns
# rename_dict = {
#     "DATE": "Date",
#     "AWND": "Avg_Wind_Speed",
#     "PRCP": "Precipitation",
#     "SNOW": "Snowfall",
#     "SNWD": "Snow_Depth",
#     "TAVG": "Avg_Temperature",
#     "TMAX": "Max_Temperature",
#     "TMIN": "Min_Temperature",
#     "WT01": "Fog",
#     "WT02": "Heavy_Fog",
#     "WT03": "Thunder",
#     "WT04": "Ice_Pellets",
#     "WT05": "Hail",
#     "WT06": "Glaze_Ice",
#     "WT08": "Smoke",
#     "WT09": "Blowing_Snow",
# }

# Apply renaming to the dataframe
# df_ord = df_ord.rename(columns=rename_dict)
# df_mdw = df_mdw.rename(columns=rename_dict)

We need to label the fields from each dataset so that when we merge them, we know where each column came from.

In [39]:
# Example for Midway (already done)
rename_ord_dict = {
    "DATE": "CRASH_DATE",
    "AWND": "Avg_Wind_Speed_ORD",
    "PRCP": "Precipitation_ORD",
    "SNOW": "Snowfall_ORD",
    "SNWD": "Snow_Depth_ORD",
    "TAVG": "Avg_Temperature_ORD",
    "TMAX": "Max_Temperature_ORD",
    "TMIN": "Min_Temperature_ORD",
    "WT01": "Fog_ORD",
    "WT02": "Heavy_Fog_ORD",
    "WT03": "Thunder_ORD",
    "WT04": "Ice_Pellets_ORD",
    "WT05": "Hail_ORD",
    "WT06": "Glaze_Ice_ORD",
    "WT08": "Smoke_ORD",
    "WT09": "Blowing_Snow_ORD",
}
rename_mdw_dict = {
    "DATE": "CRASH_DATE",
    "AWND": "Avg_Wind_Speed_MDW",
    "PRCP": "Precipitation_MDW",
    "SNOW": "Snowfall_MDW",
    "SNWD": "Snow_Depth_MDW",
    "TAVG": "Avg_Temperature_MDW",
    "TMAX": "Max_Temperature_MDW",
    "TMIN": "Min_Temperature_MDW",
    "WT01": "Fog_MDW",
    "WT02": "Heavy_Fog_MDW",
    "WT03": "Thunder_MDW",
    "WT04": "Ice_Pellets_MDW",
    "WT05": "Hail_MDW",
    "WT06": "Glaze_Ice_MDW",
    "WT08": "Smoke_MDW",
    "WT09": "Blowing_Snow_MDW",
}

# Apply renaming to the dataframe
df_ord = df_ord.rename(columns=rename_ord_dict)
df_mdw = df_mdw.rename(columns=rename_mdw_dict)


In [36]:
# view all columns in both df
#print(df_ord.head())
#print(df_mdw.head())
print(df_ord.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2557 entries, 0 to 2556
Data columns (total 46 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   STATION              2557 non-null   object 
 1   NAME                 2557 non-null   object 
 2   LATITUDE             2557 non-null   float64
 3   LONGITUDE            2557 non-null   float64
 4   ELEVATION            2557 non-null   float64
 5   CRASH_DATE           2557 non-null   object 
 6   Avg_Wind_Speed_ORD   2556 non-null   float64
 7   AWND_ATTRIBUTES      2556 non-null   object 
 8   PGTM                 35 non-null     float64
 9   PGTM_ATTRIBUTES      35 non-null     object 
 10  Precipitation_ORD    2557 non-null   float64
 11  PRCP_ATTRIBUTES      2557 non-null   object 
 12  Snowfall_ORD         2557 non-null   float64
 13  SNOW_ATTRIBUTES      2557 non-null   object 
 14  Snow_Depth_ORD       2556 non-null   float64
 15  SNWD_ATTRIBUTES      2556 non-null   o

Now let's merge both of the weather dataframes:

1. Get rid of the unnecessary data fields
2. Fill in any zeros/gaps
3. Merge

In [41]:
# 1. Get rid of the unnecessary columns
ord_keep_cols = [
    "CRASH_DATE",
    "Avg_Wind_Speed_ORD",
    "Precipitation_ORD",
    "Snowfall_ORD",
    "Snow_Depth_ORD",
    "Avg_Temperature_ORD",
    "Max_Temperature_ORD",
    "Min_Temperature_ORD",
    "Fog_ORD",
    "Heavy_Fog_ORD",
    "Thunder_ORD",
    "Ice_Pellets_ORD",
    "Hail_ORD",
    "Glaze_Ice_ORD",
    "Smoke_ORD",
    "Blowing_Snow_ORD"
]
mdw_keep_cols = [
    "CRASH_DATE",
    "Avg_Wind_Speed_MDW",
    "Precipitation_MDW",
    "Snowfall_MDW",
    "Snow_Depth_MDW",
    "Avg_Temperature_MDW",
    "Max_Temperature_MDW",
    "Min_Temperature_MDW",
    "Fog_MDW",
    "Heavy_Fog_MDW",
    "Thunder_MDW",
    "Ice_Pellets_MDW",
    "Hail_MDW",
    "Glaze_Ice_MDW",
    "Smoke_MDW",
    "Blowing_Snow_MDW"
]

# Filter only the above columns
df_ord_filtered = df_ord[ord_keep_cols]
df_mdw_filtered = df_mdw[mdw_keep_cols]

In [ ]:
# 2. Fill in any missing values

# check to see if there are any missing values
print(df_ord_filtered.isnull().sum())
print(df_mdw_filtered.isnull().sum())


# check unqiue values for each column to see what we need to fill in
weather_event_columns = ["Fog_ORD", "Heavy_Fog_ORD", "Thunder_ORD", "Ice_Pellets_ORD", "Hail_ORD", "Glaze_Ice_ORD", "Smoke_ORD", "Blowing_Snow_ORD"]

for col in weather_event_columns:
    print(f"{col}: {df_ord_filtered[col].unique()}")

## This returned all binary values, either 1 or Nan. 

# Handle missing values

# # One missing snow value... assume 0 snowfall
# df_merged["Snow_Depth"].fillna(0, inplace=True)

# # One missing windspeed value... assume 0 wind
# df_merged["Avg_Wind_Speed"].fillna(0, inplace=True)

# # The weather events are stored as a binary flag (1 = event occurred, 0 = event did not occur)
# weather_event_columns = ["Fog", "Heavy_Fog", "Thunder", "Ice_Pellets", "Hail", "Glaze_Ice", "Smoke", "Blowing_Snow"]
# for col in weather_event_columns:
#     print(f"{col}: {df_merged[col].unique()}")

# # Fill missing weather event values with 0
# df_merged[weather_event_columns] = df_merged[weather_event_columns].fillna(0)

# print(df_merged.isnull().sum()) #check that the important variables are not missing

CRASH_DATE                0
Avg_Wind_Speed_ORD        1
Precipitation_ORD         0
Snowfall_ORD              0
Snow_Depth_ORD            1
Avg_Temperature_ORD       0
Max_Temperature_ORD       0
Min_Temperature_ORD       0
Fog_ORD                1550
Heavy_Fog_ORD          2492
Thunder_ORD            2235
Ice_Pellets_ORD        2522
Hail_ORD               2535
Glaze_Ice_ORD          2516
Smoke_ORD              2120
Blowing_Snow_ORD       2529
dtype: int64
CRASH_DATE                0
Avg_Wind_Speed_MDW        1
Precipitation_MDW         4
Snowfall_MDW           2350
Snow_Depth_MDW         1963
Avg_Temperature_MDW    2557
Max_Temperature_MDW       0
Min_Temperature_MDW       0
Fog_MDW                1864
Heavy_Fog_MDW          2531
Thunder_MDW            2318
Ice_Pellets_MDW        2535
Hail_MDW               2552
Glaze_Ice_MDW          2542
Smoke_MDW              2343
Blowing_Snow_MDW       2545
dtype: int64
Fog_ORD: [nan  1.]
Heavy_Fog_ORD: [nan  1.]
Thunder_ORD: [nan  1.]
Ice_Pellets

(2557, 16)
(2557, 16)


In [ ]:
# 2. Merge the two dataframes on the CRASH_DATE column
df_weather_merged = pd.merge(df_ord_filtered, df_mdw_filtered, on="CRASH_DATE", how="inner")

Merge the two datasets (df_crashes_per_day and df_ord) based on date. 

Why are we doing this on crashes per day and not indidivual crashes?
- weather applies to the entire day, not just each individ crash
- time series works ebst with aggregated data
- simplify the dataset

In [34]:
# Ensure CRASH_DATE is in datetime format (if not already)
df['CRASH_DATE'] = pd.to_datetime(df['CRASH_DATE'])

# Aggregate total crashes per day
df_crashes_per_day = df.groupby("CRASH_DATE").size().reset_index(name="Total_Crashes")

# Display the first few rows
print(df_crashes_per_day.head())

  CRASH_DATE  Total_Crashes
0 2018-01-01            337
1 2018-01-02            452
2 2018-01-03            383
3 2018-01-04            360
4 2018-01-05            373


In [35]:
df_ord.rename(columns={"DATE": "CRASH_DATE"}, inplace=True)
#df_ord.head()
print(df_ord.columns)

df_crashes_per_day["CRASH_DATE"] = pd.to_datetime(df_crashes_per_day["CRASH_DATE"])
df_ord["CRASH_DATE"] = pd.to_datetime(df_ord["CRASH_DATE"])


# Merge crash data with weather data
df_merged = pd.merge(df_crashes_per_day, df_ord, on="CRASH_DATE", how="left")

# Verify the merged dataset
df_merged.head()




Index(['STATION', 'NAME', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'CRASH_DATE',
       'Avg_Wind_Speed', 'AWND_ATTRIBUTES', 'PGTM', 'PGTM_ATTRIBUTES',
       'Precipitation', 'PRCP_ATTRIBUTES', 'Snowfall', 'SNOW_ATTRIBUTES',
       'Snow_Depth', 'SNWD_ATTRIBUTES', 'Avg_Temperature', 'TAVG_ATTRIBUTES',
       'Max_Temperature', 'TMAX_ATTRIBUTES', 'Min_Temperature',
       'TMIN_ATTRIBUTES', 'WDF2', 'WDF2_ATTRIBUTES', 'WDF5', 'WDF5_ATTRIBUTES',
       'WSF2', 'WSF2_ATTRIBUTES', 'WSF5', 'WSF5_ATTRIBUTES', 'Fog',
       'WT01_ATTRIBUTES', 'Heavy_Fog', 'WT02_ATTRIBUTES', 'Thunder',
       'WT03_ATTRIBUTES', 'Ice_Pellets', 'WT04_ATTRIBUTES', 'Hail',
       'WT05_ATTRIBUTES', 'Glaze_Ice', 'WT06_ATTRIBUTES', 'Smoke',
       'WT08_ATTRIBUTES', 'Blowing_Snow', 'WT09_ATTRIBUTES'],
      dtype='object')


,CRASH_DATE,Total_Crashes,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,Avg_Wind_Speed,AWND_ATTRIBUTES,PGTM,...,Ice_Pellets,WT04_ATTRIBUTES,Hail,WT05_ATTRIBUTES,Glaze_Ice,WT06_ATTRIBUTES,Smoke,WT08_ATTRIBUTES,Blowing_Snow,WT09_ATTRIBUTES
0,2018-01-01,337,USW00094846,"CHICAGO OHARE INTERNATIONAL AIRPORT, IL US",41.96017,-87.93164,204.8,10.74,",,W",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-02,452,USW00094846,"CHICAGO OHARE INTERNATIONAL AIRPORT, IL US",41.96017,-87.93164,204.8,11.18,",,W",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-01-03,383,USW00094846,"CHICAGO OHARE INTERNATIONAL AIRPORT, IL US",41.96017,-87.93164,204.8,12.30,",,W",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,",,W",NaN,NaN
3,2018-01-04,360,USW00094846,"CHICAGO OHARE INTERNATIONAL AIRPORT, IL US",41.96017,-87.93164,204.8,12.30,",,W",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-01-05,373,USW00094846,"CHICAGO OHARE INTERNATIONAL AIRPORT, IL US",41.96017,-87.93164,204.8,10.74,",,W",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
df_merged.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2557 entries, 0 to 2556
Data columns (total 47 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   CRASH_DATE       2557 non-null   datetime64[ns]
 1   Total_Crashes    2557 non-null   int64         
 2   STATION          2557 non-null   object        
 3   NAME             2557 non-null   object        
 4   LATITUDE         2557 non-null   float64       
 5   LONGITUDE        2557 non-null   float64       
 6   ELEVATION        2557 non-null   float64       
 7   Avg_Wind_Speed   2556 non-null   float64       
 8   AWND_ATTRIBUTES  2556 non-null   object        
 9   PGTM             35 non-null     float64       
 10  PGTM_ATTRIBUTES  35 non-null     object        
 11  Precipitation    2557 non-null   float64       
 12  PRCP_ATTRIBUTES  2557 non-null   object        
 13  Snowfall         2557 non-null   float64       
 14  SNOW_ATTRIBUTES  2557 non-null   object 

In [37]:
# check to see if there are any missing values
print(df_merged.isnull().sum())


CRASH_DATE            0
Total_Crashes         0
STATION               0
NAME                  0
LATITUDE              0
LONGITUDE             0
ELEVATION             0
Avg_Wind_Speed        1
AWND_ATTRIBUTES       1
PGTM               2522
PGTM_ATTRIBUTES    2522
Precipitation         0
PRCP_ATTRIBUTES       0
Snowfall              0
SNOW_ATTRIBUTES       0
Snow_Depth            1
SNWD_ATTRIBUTES       1
Avg_Temperature       0
TAVG_ATTRIBUTES       0
Max_Temperature       0
TMAX_ATTRIBUTES       0
Min_Temperature       0
TMIN_ATTRIBUTES       0
WDF2                  1
WDF2_ATTRIBUTES       1
WDF5                  6
WDF5_ATTRIBUTES       6
WSF2                  1
WSF2_ATTRIBUTES       1
WSF5                  6
WSF5_ATTRIBUTES       6
Fog                1550
WT01_ATTRIBUTES    1550
Heavy_Fog          2492
WT02_ATTRIBUTES    2492
Thunder            2235
WT03_ATTRIBUTES    2235
Ice_Pellets        2522
WT04_ATTRIBUTES    2522
Hail               2535
WT05_ATTRIBUTES    2535
Glaze_Ice       

In [ ]:
# Handle missing values

# One missing snow value... assume 0 snowfall
df_merged["Snow_Depth"].fillna(0, inplace=True)

# One missing windspeed value... assume 0 wind
df_merged["Avg_Wind_Speed"].fillna(0, inplace=True)

# The weather events are stored as a binary flag (1 = event occurred, 0 = event did not occur)
weather_event_columns = ["Fog", "Heavy_Fog", "Thunder", "Ice_Pellets", "Hail", "Glaze_Ice", "Smoke", "Blowing_Snow"]
for col in weather_event_columns:
    print(f"{col}: {df_merged[col].unique()}")

# Fill missing weather event values with 0
df_merged[weather_event_columns] = df_merged[weather_event_columns].fillna(0)

print(df_merged.isnull().sum()) #check that the important variables are not missing



Fog: [0. 1.]
Heavy_Fog: [0. 1.]
Thunder: [0. 1.]
Ice_Pellets: [0. 1.]
Hail: [0. 1.]
Glaze_Ice: [0. 1.]
Smoke: [0. 1.]
Blowing_Snow: [0. 1.]
CRASH_DATE            0
Total_Crashes         0
STATION               0
NAME                  0
LATITUDE              0
LONGITUDE             0
ELEVATION             0
Avg_Wind_Speed        0
AWND_ATTRIBUTES       1
PGTM               2522
PGTM_ATTRIBUTES    2522
Precipitation         0
PRCP_ATTRIBUTES       0
Snowfall              0
SNOW_ATTRIBUTES       0
Snow_Depth            0
SNWD_ATTRIBUTES       1
Avg_Temperature       0
TAVG_ATTRIBUTES       0
Max_Temperature       0
TMAX_ATTRIBUTES       0
Min_Temperature       0
TMIN_ATTRIBUTES       0
WDF2                  1
WDF2_ATTRIBUTES       1
WDF5                  6
WDF5_ATTRIBUTES       6
WSF2                  1
WSF2_ATTRIBUTES       1
WSF5                  6
WSF5_ATTRIBUTES       6
Fog                   0
WT01_ATTRIBUTES    1550
Heavy_Fog             0
WT02_ATTRIBUTES    2492
Thunder             

C:\Users\chris\AppData\Local\Temp\ipykernel_61812\2873861122.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_merged["Snow_Depth"].fillna(0, inplace=True)
C:\Users\chris\AppData\Local\Temp\ipykernel_61812\2873861122.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exam

Set up prophet to do SARIMAX:

In [ ]:
from prophet import Prophet

# Initialize Prophet model
model = Prophet(yearly_seasonality=True, weekly_seasonality=True)

# Add external regressors (e.g., precipitation and temperature)
model.add_regressor("Precipitation")
model.add_regressor("Avg_Temperature")

# Fit the model (ensure df_prophet includes these columns)
model.fit(df_prophet)

# Make future dataframe (must include regressor values!)
future = model.make_future_dataframe(periods=365)
future["Precipitation"] = df_prophet["Precipitation"].values  # Use historical weather or predicted values
future["Avg_Temperature"] = df_prophet["Avg_Temperature"].values

# Forecast
forecast = model.predict(future)
